In [1]:
# Импорт необходимых библиотек
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re
import numpy as np
import os

In [2]:
# Проверяем доступность устройства CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [3]:
# Загрузка и нормализация данных
data_path = 'merged_data.csv'  # Укажите правильный путь к вашему файлу
data = pd.read_csv(data_path)

def normalize_text(text):
    text = text.strip().lower()
    text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
    text = re.sub(r'[^\w\s]', '', text)  # Удаление пунктуации
    return text

data["summarized_text"] = data["summarized_text"].apply(normalize_text)


In [4]:
# Кодирование меток
label_encoder = LabelEncoder()
data["encoded_tags"] = label_encoder.fit_transform(data["tags"].apply(lambda x: x.split(',')[0]))  # Одиночные метки

# Разделение данных
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Сохранение тестового набора
test_data.to_csv('test_data_tags.csv', index=False)
print("Файл 'test_data_tags.csv' успешно создан.")


Файл 'test_data_tags.csv' успешно создан.


In [5]:
# Используем модель RuBERT
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_data['summarized_text'].tolist(), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_data['summarized_text'].tolist(), truncation=True, padding=True, max_length=256)


In [6]:
class TaggingDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TaggingDataset(train_encodings, train_data['encoded_tags'].tolist())
test_dataset = TaggingDataset(test_encodings, test_data['encoded_tags'].tolist())

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))
model.to(device)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='no',
    per_device_train_batch_size=16,
    num_train_epochs=30,
    weight_decay=0.01,
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_strategy='steps',
    logging_steps=100,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/user1/.local/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
100,3.651100
200,2.947600
300,2.391900
400,1.899400
500,1.536800
600,1.277100
700,1.132700
800,0.949000
900,0.841800
1000,0.760700


TrainOutput(global_step=1530, training_loss=1.3467003379771911, metrics={'train_runtime': 142.0621, 'train_samples_per_second': 169.785, 'train_steps_per_second': 10.77, 'total_flos': 3176424185978880.0, 'train_loss': 1.3467003379771911, 'epoch': 30.0})

In [7]:
# Директория для сохранения модели
save_directory = './saved_model'
os.makedirs(save_directory, exist_ok=True)

# Сохраните классы кодировщика меток
np.save(os.path.join(save_directory, 'label_encoder_classes.npy'), label_encoder.classes_)

# Сохранение модели и токенизатора
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Модель и токенизатор сохранены в папке '{save_directory}'")

# Предсказание меток на тестовом наборе
preds_output = trainer.predict(test_dataset)
preds = np.argmax(preds_output.predictions, axis=1)

# Декодируем числовые метки в текстовые теги
predicted_tags = label_encoder.inverse_transform(preds)

# Сохранение предсказаний
test_data["predicted_tags"] = predicted_tags
sample_submission = test_data[["id", "predicted_tags"]]  # Убедитесь, что 'id' присутствует в данных
sample_submission.to_csv("sample_submission.csv", index=False)


Модель и токенизатор сохранены в папке './saved_model'


In [8]:
def iou_metric(ground_truth, predictions):
    iou = len(set.intersection(set(ground_truth), set(predictions)))
    iou = iou / len(set(ground_truth).union(set(predictions)))
    return iou

def split_tags(tag_list):
    final_tag_list = []
    for full_tag in tag_list:
        tags = full_tag.split(", ")  # Сперва разделим теги по запятым, если есть несколько тегов
        for tag in tags:
            components = tag.split(": ")
            if len(components) >= 1:
                # Поэтапно добавляем компоненты, даже если их больше трех
                combined = components[0]
                final_tag_list.append(combined)
                for comp in components[1:]:
                    combined += ": " + comp
                    final_tag_list.append(combined)
            else:
                print("NOT IMPLEMENTED!!!!", tag)
    return final_tag_list


def find_iou_for_sample_submission(pred_submission_path, true_submission_path):
    pred_submission = pd.read_csv(pred_submission_path)
    true_submission = pd.read_csv(true_submission_path)

    pred_submission["predicted_tags"] = pred_submission["predicted_tags"].apply(lambda x: [x])
    true_submission["tags"] = true_submission["tags"].apply(lambda x: [x] if x == x else [])

    true_submission["tags_split"] = true_submission["tags"].apply(lambda l: split_tags(l))
    pred_submission["predicted_tags_split"] = pred_submission["predicted_tags"].apply(lambda l: split_tags(l))

    iou = 0
    counter = 0
    for i, row in true_submission.iterrows():
        predicted_tags = pred_submission[pred_submission["id"] == row["id"]]["predicted_tags_split"].values[0]
        iou_temp = iou_metric(row['tags_split'], predicted_tags)
        iou += iou_temp
        counter += 1

    return iou/counter

# Оценка предсказаний
final_score = find_iou_for_sample_submission('sample_submission.csv', 'test_data_tags.csv')
print(f"FINAL_SCORE: {final_score:.4f}")


FINAL_SCORE: 0.3450
